<a class="reference external" href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_queryJobs_ExploreAllJobs.ipynb" target="_blank">
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Step 1: Explore All Jobs
***General Queries getJobList()***

by Silvia Mazzoni, DesignSafe, 2025

In [1]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils
import json

Using local utilities library


In [2]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-20T22:46:16+00:00
 Token expires in: 2:37:01.158774
-- LOG IN SUCCESSFUL! --


### ______________________________________________________________________________
The first step in working with Tapis jobs is to **get an overview of all your past and current runs**. Using `getJobList()`, you can retrieve summary metadata such as job IDs, owners, applications used, creation dates, and statuses. This broad view is useful for:

* Building a searchable index or DataFrame of jobs,
* Filtering by status (e.g., only finished or failed jobs), and
* Deciding which jobs to examine in more detail.

Think of this as your **dashboard view**: it gives you a wide-angle snapshot of your work across Tapis before zooming into specific jobs.

---

The **getJobList()** method retrieves a list of jobs, which can be filtered using a variety of fields. This is typically your first step when exploring job data.

**Example Use:**

In [3]:
jobs = t.jobs.getJobList()
# look at the first 3 jobs:
for job in jobs[:3]:
    print(job.uuid, job.status, job.created)
    print(job)
    print('----------')

0d3c401b-1807-45a7-904c-ea50d381d2ca-007 FAILED 2024-06-20T21:29:15.384516Z

appId: opensees-mp-s3
appVersion: 3.6.0
archiveSystemId: designsafe.storage.default
condition: None
created: 2024-06-20T21:29:15.384516Z
ended: 2024-06-20T21:40:18.629466Z
execSystemId: stampede3
lastUpdated: 2024-06-20T21:40:18.629466Z
name: opensees-mp-s3-3.6.0_2024-06-13T18:18:01
owner: silvia
remoteStarted: None
status: FAILED
tenant: designsafe
uuid: 0d3c401b-1807-45a7-904c-ea50d381d2ca-007
----------
a983892a-f8a7-45cd-91a9-fe87747bb49c-007 FAILED 2024-06-13T18:18:10.809303Z

appId: opensees-mp-s3
appVersion: 3.6.0
archiveSystemId: designsafe.storage.default
condition: None
created: 2024-06-13T18:18:10.809303Z
ended: 2024-06-13T18:19:12.957455Z
execSystemId: stampede3
lastUpdated: 2024-06-13T18:19:12.957455Z
name: opensees-mp-s3-3.6.0_2024-06-13T18:18:01
owner: silvia
remoteStarted: None
status: FAILED
tenant: designsafe
uuid: a983892a-f8a7-45cd-91a9-fe87747bb49c-007
----------
65e05dee-13d9-4b56-8d83-f9

Each job is returned as a **TapisResult object**, which can be inspected with:

In [4]:
job = jobs[0]
print(dir(job))          # All attributes
print(job.__dict__)      # Dictionary view

['PRIMITIVE_TYPES', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'appId', 'appVersion', 'archiveSystemId', 'condition', 'created', 'ended', 'execSystemId', 'get', 'lastUpdated', 'name', 'owner', 'remoteStarted', 'status', 'tenant', 'uuid']
{'uuid': '0d3c401b-1807-45a7-904c-ea50d381d2ca-007', 'name': 'opensees-mp-s3-3.6.0_2024-06-13T18:18:01', 'owner': 'silvia', 'appId': 'opensees-mp-s3', 'created': '2024-06-20T21:29:15.384516Z', 'status': 'FAILED', 'condition': None, 'remoteStarted': None, 'ended': '2024-06-20T21:40:18.629466Z', 'tenant': 'designsafe', 'execSystemId': 'stampede3', 'archiveSystemId': 'designsafe.storage.default', 'appVersion': '3.6.0', 'lastUpdated': '2024-06-2

## Queryable Fields for *getJobList()*

The *getJobList()* method does not accept all of the following fields directly, but these attributes are available on returned objects and can be filtered programmatically:

| Field             | Description                                      |
| ----------------- | ------------------------------------------------ |
| *id*              | Unique job ID                                    |
| *uuid*            | Universally unique identifier                    |
| *owner*           | Username of the job submitter                    |
| *tenant*          | Tapis tenant the job belongs to                  |
| *appId*           | Application used to launch the job               |
| *status*          | Current job status (*RUNNING*, *FINISHED*, etc.) |
| *created*         | Timestamp when job was created                   |
| *lastUpdated*     | Timestamp when job was last updated              |
| *execSystemId*    | Execution system where job ran                   |
| *archiveSystemId* | ID of archive system for outputs                 |
| *archivePath*     | Path to archived output                          |
| *parameterSet*    | Parameters and arguments submitted               |
| *tags*            | User-defined tags                                |

---

## Available Job Status Values

| Status           | Meaning               |
| ---------------- | --------------------- |
| *PENDING*        | Waiting to start      |
| *STAGING_INPUTS* | Preparing inputs      |
| *RUNNING*        | Job is active         |
| *FINISHED*       | Completed normally    |
| *FAILED*         | Failed to complete    |
| *CANCELLED*      | Cancelled by user     |
| *PAUSED*         | Paused manually       |
| *BLOCKED*        | Waiting on dependency |

Note: Some fields (e.g. *startTime*, *endTime*) may be *null* if the job hasn’t started or completed.
---

In [5]:
jobs = t.jobs.getJobList(status='FINISHED')
# look at the first 3 jobs:
for job in jobs[:3]:
    print(job.uuid, job.status, job.created)
    print(job)
    print('----------')

0d3c401b-1807-45a7-904c-ea50d381d2ca-007 FAILED 2024-06-20T21:29:15.384516Z

appId: opensees-mp-s3
appVersion: 3.6.0
archiveSystemId: designsafe.storage.default
condition: None
created: 2024-06-20T21:29:15.384516Z
ended: 2024-06-20T21:40:18.629466Z
execSystemId: stampede3
lastUpdated: 2024-06-20T21:40:18.629466Z
name: opensees-mp-s3-3.6.0_2024-06-13T18:18:01
owner: silvia
remoteStarted: None
status: FAILED
tenant: designsafe
uuid: 0d3c401b-1807-45a7-904c-ea50d381d2ca-007
----------
a983892a-f8a7-45cd-91a9-fe87747bb49c-007 FAILED 2024-06-13T18:18:10.809303Z

appId: opensees-mp-s3
appVersion: 3.6.0
archiveSystemId: designsafe.storage.default
condition: None
created: 2024-06-13T18:18:10.809303Z
ended: 2024-06-13T18:19:12.957455Z
execSystemId: stampede3
lastUpdated: 2024-06-13T18:19:12.957455Z
name: opensees-mp-s3-3.6.0_2024-06-13T18:18:01
owner: silvia
remoteStarted: None
status: FAILED
tenant: designsafe
uuid: a983892a-f8a7-45cd-91a9-fe87747bb49c-007
----------
65e05dee-13d9-4b56-8d83-f9

Or via search:

In [6]:
search_query = json.dumps({"status": "FAILED"})
jobs = t.jobs.getJobList(search=search_query)
# look at the first 3 jobs:
for job in jobs[:3]:
    print(job.uuid, job.status, job.created)
    print(job)
    print('----------')

0d3c401b-1807-45a7-904c-ea50d381d2ca-007 FAILED 2024-06-20T21:29:15.384516Z

appId: opensees-mp-s3
appVersion: 3.6.0
archiveSystemId: designsafe.storage.default
condition: None
created: 2024-06-20T21:29:15.384516Z
ended: 2024-06-20T21:40:18.629466Z
execSystemId: stampede3
lastUpdated: 2024-06-20T21:40:18.629466Z
name: opensees-mp-s3-3.6.0_2024-06-13T18:18:01
owner: silvia
remoteStarted: None
status: FAILED
tenant: designsafe
uuid: 0d3c401b-1807-45a7-904c-ea50d381d2ca-007
----------
a983892a-f8a7-45cd-91a9-fe87747bb49c-007 FAILED 2024-06-13T18:18:10.809303Z

appId: opensees-mp-s3
appVersion: 3.6.0
archiveSystemId: designsafe.storage.default
condition: None
created: 2024-06-13T18:18:10.809303Z
ended: 2024-06-13T18:19:12.957455Z
execSystemId: stampede3
lastUpdated: 2024-06-13T18:19:12.957455Z
name: opensees-mp-s3-3.6.0_2024-06-13T18:18:01
owner: silvia
remoteStarted: None
status: FAILED
tenant: designsafe
uuid: a983892a-f8a7-45cd-91a9-fe87747bb49c-007
----------
65e05dee-13d9-4b56-8d83-f9

**We will look at these commands in more details in this training module.**